In [6]:
!pip install d2l==1.0.0-beta0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Giống như các khung học sâu cấp cao giúp triển khai hồi quy tuyến tính dễ dàng hơn (xem Phần 3.5 ), chúng cũng thuận tiện tương tự ở đây.

In [7]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

# 4.5.1. Define Mô hình

Như trong Phần 3.5 , chúng tôi xây dựng lớp được kết nối đầy đủ bằng cách sử dụng lớp tích hợp. Sau đó, phương thức tích hợp `__call__` sẽ gọi forward bất cứ khi nào chúng ta cần áp dụng mạng cho một số đầu vào.

Chúng ta sử dụng tầng Flatten để chuyển đổi tenxơ bậc 4 X thành bậc 2 bằng cách giữ nguyên số chiều dọc theo trục đầu tiên.

In [8]:
class SoftmaxRegression(d2l.Classifier):
  def __init__(self, num_outputs, lr):
    super().__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(nn.Flatten(),
                             nn.LazyLinear(num_outputs))
  def forward(self,X):
    return self.net(X)

# 4.5.2. Xem lại Softmax

Trong Phần 4.4 , chúng tôi đã tính toán đầu ra của mô hình và áp dụng tổn thất entropy chéo. Mặc dù điều này hoàn toàn hợp lý về mặt toán học, nhưng nó lại rủi ro về mặt tính toán, do số bị tràn và tràn trong phép lũy thừa.

Nếu số lũy thừa tức là output tính ra quá to thì sẽ bị tràn. Giải pháp là trừ $\bar{o} \stackrel{\mathrm{def}}{=} \max_k o_k$ cho từng số hạng

$$
\hat y_j = \frac{\exp o_j}{\sum_k \exp o_k} =
\frac{\exp(o_j - \bar{o}) \exp \bar{o}}{\sum_k \exp (o_k - \bar{o}) \exp \bar{o}} =
\frac{\exp(o_j - \bar{o})}{\sum_k \exp (o_k - \bar{o})}.
$$

Khi đó, $o_j - \bar{o} \leq 0$ với mọi $j$, ngăn ngừa tràn số. Chảy tràn số xảy ra khi $\exp(o_j - \bar{o})$ đánh giá bằng số như 0. Tuy nhiên, sau một vài bước, ta có thể thấy mình phỉa đối mặt với vô số NaN.

May mắn thay, chúng tôi được cứu bởi thực tế là mặc dù chúng tôi đang tính toán các hàm số mũ, nhưng cuối cùng chúng tôi vẫn có ý định lấy logarit của chúng (khi tính toán tổn thất entropy chéo). Bằng cách kết hợp softmax và cross-entropy, chúng ta có thể thoát khỏi hoàn toàn các vấn đề về ổn định số. Chúng ta có:

$$
\log \hat{y}_j =
\log \frac{\exp(o_j - \bar{o})}{\sum_k \exp (o_k - \bar{o})} =
o_j - \bar{o} - \log \sum_k \exp (o_k - \bar{o}).
$$

Điều này tránh cả tràn và tràn. Chúng tôi sẽ muốn giữ chức năng softmax thông thường tiện dụng trong trường hợp chúng tôi muốn đánh giá xác suất đầu ra theo mô hình của mình. Nhưng thay vì chuyển xác suất softmax vào hàm mất mát mới của chúng tôi, chúng tôi chỉ chuyển log và tính toán softmax và log của nó cùng một lúc bên trong hàm mất mát entropy chéo, hàm này thực hiện những điều thông minh như “thủ thuật LogSumExp ” .

In [9]:
@d2l.add_to_class(d2l.Classifier)
def loss(self, Y_hat, Y, averaged=True):
  Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
  Y = Y.reshape((-1,))
  return F.cross_entropy(
      Y_hat, Y, reduction='mean' if averaged else 'none'
  )

# 4.5.3. Training

Tiếp theo, ta sẽ training model. Chúng ta sử dụng hình ảnh Fashion-MNIST, được làm phẳng thành các vectơ đặc trưng 784 chiều.

In [13]:
data = d2l.FashionMNIST(batch_size=256)
X, y = next(iter(data.train_dataloader()))
print(X.shape, y.shape)
model = SoftmaxRegression(num_outputs=10, lr=0.1)
trainer = d2l.Trainer(max_epochs=10)
#trainer.fit(model, data)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([256, 1, 28, 28]) torch.Size([256])


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
